# Stage 04: Data Acquisition and Ingestion


## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [11]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

RAW = pathlib.Path('data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? True


## Helpers (use or modify)

In [12]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [6]:
SYMBOL = 'AAPL'
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))
if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {'function':'TIME_SERIES_DAILY_ADJUSTED','symbol':SYMBOL,'outputsize':'compact','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    key = [k for k in js if 'Time Series' in k][0]
    df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date','5. adjusted close':'adj_close'})[['date','adj_close']]
    df_api['date'] = pd.to_datetime(df_api['date']); df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
else:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Close']]
    df_api.columns = ['date','adj_close']

v_api = validate(df_api, ['date','adj_close']); v_api

/var/folders/xt/g4dt2xmn47x1nq6hz8n9pvl80000gn/T/ipykernel_52220/700782901.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Close']]
[*********************100%***********************]  1 of 1 completed


{'missing': [], 'shape': (64, 2), 'na_total': 0}

In [7]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

Saved data/raw/api_source-yfinance_symbol-AAPL_20250820-102754.csv


## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [23]:
SCRAPE_URL = 'https://en.wikipedia.org/wiki/Nasdaq-100' 
headers = {'User-Agent':'AFE-Homework/1.0'}
try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30); resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    tables = soup.find_all('table', {'class': 'wikitable'})
    table = tables[2]
    header = [th.get_text(strip=True) for th in table.find('tr').find_all('th')]
    data = []
    for tr in table.find_all('tr')[1:]:  
        cells = [td.get_text(strip=True) for td in tr.find_all('td')]
        if len(cells) == len(header):
            data.append(cells)
    df_scrape = pd.DataFrame(data, columns=header)
    
except Exception as e:
    print('Scrape failed, using inline demo table:', e)
    html = '<table><tr><th>Ticker</th><th>Price</th></tr><tr><td>AAA</td><td>101.2</td></tr></table>'
    soup = BeautifulSoup(html, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

v_scrape = validate(df_scrape, list(df_scrape.columns))
print(v_scrape)

df_scrape.head()

{'missing': [], 'shape': (40, 4), 'na_total': 0}


,Year,Closing level,Change in Indexin Points,Change in Indexin %
0,1985,132.29,,
1,1986,141.41,9.12,6.89
2,1987,156.25,14.84,10.49
3,1988,177.41,21.16,13.54
4,1989,223.84,46.43,26.17


In [24]:
_ = save_csv(df_scrape, prefix='scrape', site='example', table='markets')

Saved data/raw/scrape_site-example_table-markets_20250820-175449.csv


## Documentation
- API Source: (URL/endpoint/params)
- Scrape Source: (URL/table description)
- Assumptions & risks: (rate limits, selector fragility, schema changes)
- Confirm `.env` is not committed.

In [ ]:
### Data Sources
- API (yfinance fallback)
- Scrape: Wikipedia Nasdaq-100 constituents table (id="constituents")

### Validation
- Check required columns exist (['date','adj_close'] for API; full header for scrape)
- Count NA totals
- Print df.shape

### Assumptions & Risks
- API rate limits; schema may change
- HTML table structure may change; use fallback demo table
- .env is local only (not committed)

### .env
- Confirmed .env is not committed; .env.example is in repo.